In [ ]:
import numpy as np

In [ ]:
%load_ext cython

In [ ]:
%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
import numpy as np
cimport numpy as np
from cython.parallel import prange,  parallel
@cython.boundscheck(False)
@cython.wraparound(False)
def check_bench(np.double_t[:] data):
    cdef Py_ssize_t i
    cdef int j = len(data)
    cdef double k = 6
    for i in prange(data.shape[0], nogil=True):
        data[i] = data[i] * data[i]

In [ ]:
def check_bench_2(data):
    for i in range(len(data)):
        data[i] = data[i] * 6

In [ ]:
import timeit

In [ ]:
tdata = np.array(range(300_000_000),dtype=np.double)

In [ ]:
%timeit -r 20 -n 1 check_bench(tdata)

In [ ]:
%timeit -r 1 -n 1 check_bench_2(tdata)

In [ ]:
%%file test.c
#include "check.h"
int check_bench(double* data, int n)
{
    int i = 0;
    #pragma omp parallel for 
    for (i = 0;i < n; i++)
    {
        data[i] = data[i] * data[i];
    }
    return 0;
}

In [ ]:
%%file check.h
#include<stdio.h>
int check_bench(double* , int n);

In [ ]:
%%file check.pyx 
cimport numpy as np
cimport cython
#cython: language_level=3
# cython: language_level=3, boundscheck=False
# cdefine the signature of our c function

cdef extern from "check.h":
    int  check_bench(double* a, int b)

def check(np.ndarray[double, ndim=1, mode="c"] a not None, int n):
    return check_bench(<double*> np.PyArray_DATA(a), n)

In [ ]:
%%file setup.py
from setuptools import setup
from distutils.core import Extension
from Cython.Distutils import build_ext
from Cython.Build import cythonize
import numpy
import cython

extentions = [Extension("bex",
                           sources=["check.pyx", "test.c"],
                           language='c',
                           include_dirs=[numpy.get_include()],
                           extra_compile_args=["-Wall","/openmp"],
                           extra_link_args=["/openmp"],
                          )]
 
setup(
    compiler="gcc",
    cmdclass={'build_ext': build_ext},
    ext_modules=extentions
)

In [ ]:
 ! python setup.py build_ext --inplace --force 

In [ ]:
import bex as bx

In [ ]:
%timeit -r 100 -n 1 bx.check(tdata, len(tdata))